<a href="https://colab.research.google.com/github/gtbnhyujmj/Auto_Shorts_Maker/blob/main/Movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import cv2
import numpy as np
from PIL import Image
import os

# 定義函數

In [23]:
def overlay_png_on_video(video_path, png_path, output_path, position=(0, 0)):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("無法打開影片")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    # 讀取 PNG 並轉為帶透明度的格式
    overlay_img = Image.open(png_path).convert("RGBA").resize((w, h))  # 你可以改尺寸或用原圖
    overlay_np = np.array(overlay_img)

    # 分離圖層
    overlay_rgb = overlay_np[..., :3]
    overlay_alpha = overlay_np[..., 3:] / 255.0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 將 BGR → RGB，轉成 PIL 處理格式
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_np = frame_rgb.astype(float)

        # 疊圖（透明度混合）
        blended = frame_np * (1 - overlay_alpha) + overlay_rgb * overlay_alpha
        blended = np.clip(blended, 0, 255).astype(np.uint8)

        # 轉回 BGR 寫入影片
        blended_bgr = cv2.cvtColor(blended, cv2.COLOR_RGB2BGR)
        out.write(blended_bgr)

    cap.release()
    out.release()
    print("完成輸出：", output_path)

# 設定資料夾路徑

In [24]:
# In Google Drive

bg_folder = "/content/drive/MyDrive/shorts/backgrounds_MP4/bg1.mp4"
char_folder = "/content/drive/MyDrive/shorts/characters/char1.png"
overlay_folder = "/content/drive/MyDrive/shorts/PNG_Overlayers/OverLayers1.png"

# 執行區域

In [25]:
output_path = "/content/BBB.mp4"

In [26]:
# bg_folder
# char_folder
# overlay_folder

overlay_png_on_video(char_folder, overlay_folder, output_path, position=(0, 0))

完成輸出： /content/BBB.mp4


# 檢討

In [27]:
# 他這個做法好像不能循環兩次，背景會不見。
# 因為這段程式對每一幀都直接用透明 PNG 疊圖，而沒有正確地處理 Alpha 混合（透明度累積）。
# 如果 PNG 中的 alpha 是半透明或透明，長時間疊下來會「越來越黑」或「越來越透明」。

# 在處理

In [ ]:
# STU